In [1]:
"""
FOOD DETECTION
================================================================


"""

# =============================================================================
# Mount Google Drive
# =============================================================================

from google.colab import drive
drive.mount('/content/drive', force_remount=True)
print("Google Drive mounted!")


# =============================================================================
# Smart Dataset Setup (Downloads only if not in Drive)
# =============================================================================

import os
from pathlib import Path
import shutil

# Google Drive paths
DRIVE_DATA_DIR = Path("/content/drive/MyDrive/AAI521-Final-Project/data/food-101")
IMAGES_DIR = DRIVE_DATA_DIR / "images"
META_DIR = DRIVE_DATA_DIR / "meta"

# Check if dataset exists
if IMAGES_DIR.exists() and META_DIR.exists():
    print("Dataset found in Google Drive - using existing copy!")
    print(f"   Classes: {len([d for d in IMAGES_DIR.iterdir() if d.is_dir()])}")
else:
    print("First time - downloading dataset (10-15 min, only once)...")

    import kagglehub
    cache_path = kagglehub.dataset_download("dansbecker/food-101")

    # Find dataset in cache
    cache_base = Path(cache_path)
    images_dirs = list(cache_base.rglob("images"))

    if images_dirs:
        source_images = images_dirs[0]
        source_meta = source_images.parent / "meta"

        # Copy to Google Drive
        DRIVE_DATA_DIR.mkdir(parents=True, exist_ok=True)
        print("Copying to Google Drive...")
        shutil.copytree(source_images, IMAGES_DIR)
        shutil.copytree(source_meta, META_DIR)
        print("Dataset saved to Google Drive!")
    else:
        raise FileNotFoundError("Could not find dataset")

print(f"Ready! Images: {IMAGES_DIR}")


# =============================================================================
# Install & Import Libraries
# =============================================================================

!pip install -q tensorflow opencv-python scikit-learn

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import json
import warnings
from collections import Counter
warnings.filterwarnings('ignore')

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

print("TensorFlow:", tf.__version__)
print("GPU:", len(tf.config.list_physical_devices('GPU')) > 0)
tf.random.set_seed(42)
np.random.seed(42)
print("Libraries loaded!")


# =============================================================================
# Configuration
# =============================================================================

SAVE_DIR = Path("/content/drive/MyDrive/AAI521-Final-Project/outputs")
SAVE_DIR.mkdir(parents=True, exist_ok=True)

IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS_CNN = 20
EPOCHS_FINETUNE = 10
LEARNING_RATE = 0.001
SEED = 42

print(f"Outputs will be saved to: {SAVE_DIR}")


# =============================================================================
# Calorie Database
# =============================================================================

CALORIE_DB = {
    'apple_pie': 237, 'baby_back_ribs': 361, 'baklava': 428, 'beef_carpaccio': 120,
    'beef_tartare': 180, 'beet_salad': 89, 'beignets': 303, 'bibimbap': 490,
    'bread_pudding': 310, 'breakfast_burrito': 350, 'bruschetta': 109, 'caesar_salad': 184,
    'cannoli': 300, 'caprese_salad': 108, 'carrot_cake': 435, 'ceviche': 120,
    'cheesecake': 321, 'cheese_plate': 150, 'chicken_curry': 250, 'chicken_quesadilla': 529,
    'chicken_wings': 290, 'chocolate_cake': 352, 'chocolate_mousse': 225, 'churros': 320,
    'clam_chowder': 236, 'club_sandwich': 590, 'crab_cakes': 160, 'creme_brulee': 294,
    'croque_madame': 456, 'cup_cakes': 305, 'deviled_eggs': 62, 'donuts': 269,
    'dumplings': 41, 'edamame': 122, 'eggs_benedict': 450, 'escargots': 142,
    'falafel': 333, 'filet_mignon': 227, 'fish_and_chips': 585, 'foie_gras': 462,
    'french_fries': 312, 'french_onion_soup': 330, 'french_toast': 220, 'fried_calamari': 175,
    'fried_rice': 228, 'frozen_yogurt': 127, 'garlic_bread': 150, 'gnocchi': 250,
    'greek_salad': 106, 'grilled_cheese_sandwich': 440, 'grilled_salmon': 367, 'guacamole': 150,
    'gyoza': 64, 'hamburger': 354, 'hot_and_sour_soup': 91, 'hot_dog': 290,
    'huevos_rancheros': 400, 'hummus': 166, 'ice_cream': 207, 'lasagna': 378,
    'lobster_bisque': 200, 'lobster_roll_sandwich': 436, 'macaroni_and_cheese': 320,
    'macarons': 95, 'miso_soup': 40, 'mussels': 86, 'nachos': 346,
    'omelette': 154, 'onion_rings': 276, 'oysters': 68, 'pad_thai': 380,
    'paella': 372, 'pancakes': 227, 'panna_cotta': 250, 'peking_duck': 337,
    'pho': 350, 'pizza': 266, 'pork_chop': 231, 'poutine': 740,
    'prime_rib': 361, 'pulled_pork_sandwich': 450, 'ramen': 436, 'ravioli': 220,
    'red_velvet_cake': 478, 'risotto': 320, 'samosa': 262, 'sashimi': 127,
    'scallops': 137, 'seaweed_salad': 45, 'shrimp_and_grits': 363, 'spaghetti_bolognese': 350,
    'spaghetti_carbonara': 400, 'spring_rolls': 140, 'steak': 271, 'strawberry_shortcake': 350,
    'sushi': 143, 'tacos': 226, 'takoyaki': 103, 'tiramisu': 240,
    'tuna_tartare': 145, 'waffles': 291
}

print(f"Calorie database: {len(CALORIE_DB)} foods")


# =============================================================================
# Load Dataset
# =============================================================================

def read_list(p):
    with open(p, 'r') as f:
        return [ln.strip() for ln in f if ln.strip()]

train_list = read_list(META_DIR / "train.txt")
test_list = read_list(META_DIR / "test.txt")

class_names = sorted([p.name for p in IMAGES_DIR.iterdir() if p.is_dir()])
class_to_idx = {c: i for i, c in enumerate(class_names)}
num_classes = len(class_names)

train_classes = [s.split("/")[0] for s in train_list]
train_counts = Counter(train_classes)

print("=" * 60)
print(f"Training samples: {len(train_list)}")
print(f"Test samples: {len(test_list)}")
print(f"Classes: {num_classes}")
print(f"Per class: {len(train_list) // num_classes} train, {len(test_list) // num_classes} test")

# Visualize distribution
plt.figure(figsize=(15, 4))
plt.bar(range(len(train_counts)), train_counts.values())
plt.axhline(np.mean(list(train_counts.values())), color='r', linestyle='--')
plt.xlabel('Class')
plt.ylabel('Samples')
plt.title('Training Data Distribution')
plt.savefig(SAVE_DIR / 'class_distribution.png', dpi=100, bbox_inches='tight')
plt.show()


# =============================================================================
# Create Data Pipeline
# =============================================================================

def create_path_label_pairs(file_list):
    pairs = []
    for stem in file_list:
        cls, base = stem.split("/", 1)
        path = str(IMAGES_DIR / cls / f"{base}.jpg")
        label = class_to_idx[cls]
        pairs.append((path, label))
    return pairs

train_pairs = create_path_label_pairs(train_list)
test_pairs = create_path_label_pairs(test_list)

def decode_resize(path, label):
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, IMG_SIZE)
    img = preprocess_input(img)
    return img, label

augment = keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.15),
    layers.RandomZoom(0.15),
    layers.RandomContrast(0.15),
    layers.RandomBrightness(0.15),
], name="augmentation")

train_paths, train_labels = zip(*train_pairs)
test_paths, test_labels = zip(*test_pairs)

train_ds = (tf.data.Dataset
           .from_tensor_slices((list(train_paths), list(train_labels)))
           .shuffle(len(train_pairs), seed=SEED)
           .map(decode_resize, num_parallel_calls=tf.data.AUTOTUNE)
           .batch(BATCH_SIZE)
           .prefetch(tf.data.AUTOTUNE))

train_ds = train_ds.map(lambda img, label: (augment(img, training=True), label))

test_ds = (tf.data.Dataset
          .from_tensor_slices((list(test_paths), list(test_labels)))
          .map(decode_resize, num_parallel_calls=tf.data.AUTOTUNE)
          .batch(BATCH_SIZE)
          .prefetch(tf.data.AUTOTUNE))

print(f"Train batches: {len(train_ds)}, Test batches: {len(test_ds)}")

# Show samples
for images, labels in train_ds.take(1):
    plt.figure(figsize=(12, 12))
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        img = (images[i].numpy() + 1) / 2.0
        plt.imshow(img)
        plt.title(class_names[labels[i].numpy()].replace('_', ' '), fontsize=9)
        plt.axis("off")
    plt.suptitle('Sample Training Images')
    plt.savefig(SAVE_DIR / 'sample_images.png', dpi=100, bbox_inches='tight')
    plt.show()


# =============================================================================
# Build CNN Model (Custom Top-5 Metric)
# =============================================================================

# Custom Top-5 Accuracy Metric that works with sparse labels
class SparseTop5CategoricalAccuracy(keras.metrics.Metric):
    def __init__(self, name='top5_acc', **kwargs):
        super().__init__(name=name, **kwargs)
        self.total = self.add_weight(name='total', initializer='zeros')
        self.count = self.add_weight(name='count', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        # y_true shape: (batch_size,) - sparse labels
        # y_pred shape: (batch_size, num_classes) - predictions

        # Get top 5 predictions
        top5_pred = tf.math.top_k(y_pred, k=5).indices

        # Expand y_true to match shape for comparison
        y_true_expanded = tf.expand_dims(tf.cast(y_true, tf.int32), axis=-1)

        # Check if true label is in top 5
        matches = tf.reduce_any(tf.equal(top5_pred, y_true_expanded), axis=-1)
        matches = tf.cast(matches, tf.float32)

        if sample_weight is not None:
            matches = matches * sample_weight
            self.count.assign_add(tf.reduce_sum(sample_weight))
        else:
            self.count.assign_add(tf.cast(tf.shape(y_true)[0], tf.float32))

        self.total.assign_add(tf.reduce_sum(matches))

    def result(self):
        return self.total / self.count

    def reset_state(self):
        self.total.assign(0.)
        self.count.assign(0.)


def build_cnn():
    inputs = layers.Input(shape=(*IMG_SIZE, 3))
    base_model = MobileNetV2(include_top=False, weights='imagenet', input_shape=(*IMG_SIZE, 3))
    base_model.trainable = False

    x = base_model(inputs, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(512, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    model = keras.Model(inputs, outputs)
    model.compile(
        optimizer=keras.optimizers.Adam(LEARNING_RATE),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy', SparseTop5CategoricalAccuracy()]  # FIXED: Using custom metric
    )
    return model, base_model

cnn_model, base_model = build_cnn()
print("CNN model built with fixed Top-5 metric")
cnn_model.summary()


# =============================================================================
# Train CNN
# =============================================================================

callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-7),
    ModelCheckpoint(str(SAVE_DIR / 'best_cnn_stage1.keras'), monitor='val_accuracy', save_best_only=True)
]

print("Training CNN Stage 1 (frozen base)...")
history = cnn_model.fit(
    train_ds,
    validation_data=test_ds,
    epochs=EPOCHS_CNN,
    callbacks=callbacks,
    verbose=1
)

# Plot
fig, axes = plt.subplots(1, 2, figsize=(14, 4))
axes[0].plot(history.history['accuracy'], label='Train')
axes[0].plot(history.history['val_accuracy'], label='Val')
axes[0].set_title('Accuracy')
axes[0].legend()
axes[0].grid(alpha=0.3)

axes[1].plot(history.history['loss'], label='Train')
axes[1].plot(history.history['val_loss'], label='Val')
axes[1].set_title('Loss')
axes[1].legend()
axes[1].grid(alpha=0.3)

plt.suptitle('Stage 1 Training')
plt.savefig(SAVE_DIR / 'training_stage1.png', dpi=100, bbox_inches='tight')
plt.show()
print("Stage 1 complete!")


# =============================================================================
# Fine-tune CNN
# =============================================================================

print("Fine-tuning CNN (unfreezing top layers)...")
base_model.trainable = True
for layer in base_model.layers[:-30]:
    layer.trainable = False

cnn_model.compile(
    optimizer=keras.optimizers.Adam(LEARNING_RATE / 10),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy', SparseTop5CategoricalAccuracy()]  # FIXED: Using custom metric
)

callbacks_ft = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ModelCheckpoint(str(SAVE_DIR / 'best_cnn_finetuned.keras'), monitor='val_accuracy', save_best_only=True)
]

history_ft = cnn_model.fit(
    train_ds,
    validation_data=test_ds,
    epochs=EPOCHS_FINETUNE,
    callbacks=callbacks_ft,
    verbose=1
)

# Plot
fig, axes = plt.subplots(1, 2, figsize=(14, 4))
axes[0].plot(history_ft.history['accuracy'], label='Train')
axes[0].plot(history_ft.history['val_accuracy'], label='Val')
axes[0].set_title('Accuracy')
axes[0].legend()
axes[0].grid(alpha=0.3)

axes[1].plot(history_ft.history['loss'], label='Train')
axes[1].plot(history_ft.history['val_loss'], label='Val')
axes[1].set_title('Loss')
axes[1].legend()
axes[1].grid(alpha=0.3)

plt.suptitle('Stage 2 Fine-tuning')
plt.savefig(SAVE_DIR / 'training_stage2.png', dpi=100, bbox_inches='tight')
plt.show()
print("Fine-tuning complete!")


# =============================================================================
# Evaluate CNN
# =============================================================================

print("Evaluating model...")
results = cnn_model.evaluate(test_ds, verbose=0)
metrics = dict(zip(cnn_model.metrics_names, results))

print("\n" + "=" * 60)
print("FINAL TEST RESULTS")
print("=" * 60)
for metric, value in metrics.items():
    print(f"{metric}: {value:.4f}")

# Visualize predictions
images, labels = next(iter(test_ds.take(1)))
predictions = cnn_model.predict(images[:16], verbose=0)

fig, axes = plt.subplots(4, 4, figsize=(14, 14))
for i, ax in enumerate(axes.flat):
    img = (images[i].numpy() + 1) / 2.0
    true_label = class_names[labels[i].numpy()]
    pred_label = class_names[np.argmax(predictions[i])]
    conf = np.max(predictions[i])

    color = 'green' if true_label == pred_label else 'red'
    ax.imshow(img)
    ax.set_title(f'True: {true_label}\nPred: {pred_label}\n{conf:.2f}', color=color, fontsize=8)
    ax.axis('off')

plt.suptitle('Model Predictions')
plt.savefig(SAVE_DIR / 'predictions.png', dpi=100, bbox_inches='tight')
plt.show()


# =============================================================================
# Prediction Function
# =============================================================================

def predict_food(image_path, model, top_k=5):
    """Predict food and calories"""
    img = tf.io.read_file(str(image_path))
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, IMG_SIZE)
    img = preprocess_input(img)
    img = tf.expand_dims(img, 0)

    predictions = model.predict(img, verbose=0)
    top_indices = np.argsort(predictions[0])[-top_k:][::-1]

    results = []
    for idx in top_indices:
        food_name = class_names[idx]
        confidence = predictions[0][idx]
        calories = CALORIE_DB.get(food_name, "Unknown")
        results.append({
            'food': food_name.replace('_', ' ').title(),
            'confidence': f"{confidence * 100:.2f}%",
            'calories': calories
        })
    return results

print("Prediction function ready!")
print("\nUsage: results = predict_food('image.jpg', cnn_model)")


# =============================================================================
# Test Prediction
# =============================================================================

# Get a test image
test_images, test_labels = next(iter(test_ds.take(1)))
sample_idx = 0

# Save temp image
sample_img = test_images[sample_idx]
sample_img_uint8 = tf.cast((sample_img + 1) * 127.5, tf.uint8)
temp_path = SAVE_DIR / 'temp_test.jpg'
tf.io.write_file(str(temp_path), tf.image.encode_jpeg(sample_img_uint8))

# Predict
results = predict_food(temp_path, cnn_model)

print("\n" + "=" * 60)
print("PREDICTION DEMO")
print("=" * 60)
print(f"Actual: {class_names[test_labels[sample_idx].numpy()]}\n")
for i, r in enumerate(results, 1):
    print(f"{i}. {r['food']}")
    print(f"   Confidence: {r['confidence']}")
    print(f"   Calories: {r['calories']} kcal/100g\n")

# Visualize
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
ax1.imshow((test_images[sample_idx].numpy() + 1) / 2.0)
ax1.axis('off')
ax1.set_title('Test Image')

ax2.axis('off')
top = results[0]
text = f"{top['food']}\n\n"
text += f"Confidence: {top['confidence']}\n"
text += f"Calories: {top['calories']} kcal/100g"
ax2.text(0.5, 0.7, text, ha='center', va='center', fontsize=14,
         bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.8))

others = "\n".join([f"{i}. {r['food']} ({r['confidence']})"
                     for i, r in enumerate(results[1:], 2)])
ax2.text(0.5, 0.3, others, ha='center', va='top', fontsize=10,
         bbox=dict(boxstyle='round', facecolor='lightyellow', alpha=0.8))

plt.savefig(SAVE_DIR / 'prediction_demo.png', dpi=100, bbox_inches='tight')
plt.show()


# =============================================================================
# Save Results
# =============================================================================

summary = {
    'dataset': {
        'classes': num_classes,
        'train_samples': len(train_list),
        'test_samples': len(test_list)
    },
    'performance': {
        'test_loss': float(metrics['loss']),
        'test_accuracy': float(metrics['accuracy']),
        'test_top5_accuracy': float(metrics['top5_acc'])
    },
    'config': {
        'img_size': IMG_SIZE,
        'batch_size': BATCH_SIZE,
        'epochs_cnn': EPOCHS_CNN,
        'epochs_finetune': EPOCHS_FINETUNE
    }
}

with open(SAVE_DIR / 'results.json', 'w') as f:
    json.dump(summary, f, indent=2)

print("\n" + "=" * 60)
print("PROJECT COMPLETE!")
print("=" * 60)
print(f"\nAll files saved to: {SAVE_DIR}")
print("\nGenerated files:")
print("  best_cnn_stage1.keras")
print("  best_cnn_finetuned.keras (← Use this for predictions)")
print("  class_distribution.png")
print("  sample_images.png")
print("  training_stage1.png")
print("  training_stage2.png")
print("  predictions.png")
print("  prediction_demo.png")
print("  results.json")



MessageError: Error: credential propagation was unsuccessful